In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as tt
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import matplotlib
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
import os
import tarfile
%matplotlib inline

matplotlib.rcParams['figure.facecolor'] = '#ffffff'

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd /content/gdrive/MyDrive/RockClassification

/content/gdrive/MyDrive/RockClassification


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/RockClassification"

In [ ]:
# !kaggle datasets download -d salmaneunus/rock-classification

In [ ]:
# from zipfile import ZipFile

In [ ]:
# with ZipFile("rock-classification.zip", 'r') as zip:
#     # printing all the contents of the zip file
#     zip.printdir()
  
#     # extracting all the files
#     print('Extracting all the files now...')
#     zip.extractall()
#     print('Done!')

In [ ]:
data_dir = "/content/gdrive/MyDrive/RockClassification/Dataset"
print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
from PIL import Image
import glob

basewidth = 300
input_dim = (32, 32)
# for cl in classes:
#   i=0
#   for name in glob.glob(data_dir+"/train/"+cl+"/*"):
#     print(name)
#     img = Image.open(name)
#     # wpercent = (basewidth / float(img.size[0]))
#     # hsize = int((float(img.size[1]) * float(wpercent)))
#     img = img.resize(input_dim, Image.ANTIALIAS)
#     print(img.size)
#     img.convert('RGB').save(name)
#     # print("Saving Image")
#     i += 1

# for cl in classes:
#   i=0
#   for name in glob.glob(data_dir+"/test/"+cl+"/*"):
#     print(name)
#     img = Image.open(name)
#     # wpercent = (basewidth / float(img.size[0]))
#     # hsize = int((float(img.size[1]) * float(wpercent)))
#     img = img.resize(input_dim, Image.ANTIALIAS)
#     print(img.size)
#     img.convert('RGB').save(name)
#     # print("Saving Image")
#     i += 1

['test', 'train', 'Igneous', 'Metamorphic', 'Sedimentary']


In [ ]:
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
train_tfms = tt.Compose([tt.RandomCrop(32, padding=4, padding_mode='reflect'), 
                         tt.RandomHorizontalFlip(), 
                         # tt.RandomRotate
                         # tt.RandomResizedCrop(256, scale=(0.5,0.9), ratio=(1, 1)), 
                         # tt.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
                         tt.ToTensor(), 
                         tt.Normalize(*stats,inplace=True)])
valid_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*stats)])

In [ ]:
train_ds = ImageFolder(data_dir+'/train', train_tfms)
valid_ds = ImageFolder(data_dir+'/test', valid_tfms)

In [ ]:
batch_size = 32

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

device = get_default_device()

print(device)

cuda


In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
def denormalize(images, means, stds):
    means = torch.tensor(means).reshape(1, 3, 1, 1)
    stds = torch.tensor(stds).reshape(1, 3, 1, 1)
    return images * stds + means

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.set_xticks([]); ax.set_yticks([])
        denorm_images = denormalize(images, *stats)
        ax.imshow(make_grid(denorm_images[:64], nrow=8).permute(1, 2, 0).clamp(0,1))
        break

In [ ]:
train_dl = DataLoader(train_ds, batch_size, num_workers=2, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size*2, shuffle=True, num_workers=2, pin_memory=True)
train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)

In [ ]:
# show_batch(train_dl)

In [ ]:
class SimpleResidualBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, stride =1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, stride =1)
        self.relu2 = nn.ReLU()
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        return self.relu2(out) + x

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels))/len(preds)

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images ,  labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss
    def validation_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {'val_loss': loss.detach(), 'val_acc': acc}
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accs = [x['val_acc'] for x in outputs]
        # batch_accs = torch.FloatTensor(batch_accs)
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss' : epoch_loss.item(), 'val_acc' : epoch_acc.item()}
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

In [ ]:
def conv_block(in_channels, out_channels, pool = False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding=1),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace = True)]
    if pool:
        layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)
class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block(128,128), conv_block(128,128))
        
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block(512,512), conv_block(512,512))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                        nn.Flatten(),
                                       nn.Dropout(0.2),
                                       nn.Linear(512, num_classes))
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)

        out  = self.res1(out) + out

        out = self.conv3(out)

        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [ ]:
print(len(classes))

model = ResNet9(3,len(classes))

7


In [ ]:
model(torch.zeros(1,3,32,32))

tensor([[ 2.2057,  2.9941,  0.3096, -3.9334,  2.5692,  2.3175, -2.0911]],
       grad_fn=<AddmmBackward0>)

In [ ]:
to_device(model,  device)

ResNet9(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=Tr

In [ ]:
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [ ]:
def get_lr(optimizer):
    for param_group in  optimizer.param_groups:
        return param_group['lr']

In [ ]:
def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    for epoch in range(epochs):
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            optimizer.step()
            optimizer.zero_grad()
            
            lrs.append(get_lr(optimizer))
            sched.step
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
        
    return history
        

In [ ]:
epochs = 100
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [ ]:
history = fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl, 
                             weight_decay=weight_decay, grad_clip=grad_clip, 
                             opt_func=opt_func)

<ipython-input-19-e74db46226ba>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(torch.sum(preds == labels))/len(preds)


Epoch [0], last_lr: 0.00040, train_loss: 4.0654, val_loss: 4.1481, val_acc: 0.0938
Epoch [1], last_lr: 0.00040, train_loss: 4.1172, val_loss: 4.2926, val_acc: 0.0938
Epoch [2], last_lr: 0.00040, train_loss: 4.0822, val_loss: 4.0998, val_acc: 0.1328
Epoch [3], last_lr: 0.00040, train_loss: 4.0416, val_loss: 4.2711, val_acc: 0.1328
Epoch [4], last_lr: 0.00040, train_loss: 4.0604, val_loss: 4.2929, val_acc: 0.0938
Epoch [5], last_lr: 0.00040, train_loss: 4.0117, val_loss: 4.0907, val_acc: 0.0911
Epoch [6], last_lr: 0.00040, train_loss: 4.1280, val_loss: 3.9680, val_acc: 0.1328
Epoch [7], last_lr: 0.00040, train_loss: 4.1001, val_loss: 4.0276, val_acc: 0.0938
Epoch [8], last_lr: 0.00040, train_loss: 4.0889, val_loss: 5.0971, val_acc: 0.0938
Epoch [9], last_lr: 0.00040, train_loss: 4.1073, val_loss: 4.2049, val_acc: 0.0938
Epoch [10], last_lr: 0.00040, train_loss: 4.0809, val_loss: 4.1776, val_acc: 0.1328
Epoch [11], last_lr: 0.00040, train_loss: 4.1064, val_loss: 4.3510, val_acc: 0.1354
Ep

KeyboardInterrupt: ignored